In [ ]:
pip install duckduckgo_search
pip install fastcore
pip install fastdownload
pip install fastai
print("It Begins");

In [13]:
import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")


In [14]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

In [16]:
from fastai.vision.all import *

In [18]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [19]:
urls = search_images('bird photos', max_images=1)
urls[0]

In [20]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [21]:
download_url(search_images('forest photos', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

In [22]:
searches = 'forest','bird'
path = Path('bird_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)


### Train the model

In [23]:
# To remove broken photos
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [24]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32,num_workers=0)

dls.show_batch(max_n=6)

In [25]:
blocks=(ImageBlock, CategoryBlock),


In [26]:
get_items=get_image_files, 


In [27]:
splitter=RandomSplitter(valid_pct=0.2, seed=42),

In [28]:
get_y=parent_label,


In [29]:
item_tfms=[Resize(192, method='squish')]

In [30]:
learn = vision_learner(dls, arch=resnet34, metrics=error_rate)
learn.fine_tune(4)

### Predicting the model

In [ ]:
from PIL import Image
  
# open method used to open different extension image file
im = Image.open('bird.jpg') 
  
# This method will show image in any image viewer 
im.show()

In [31]:
is_bird,_,probs = learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {is_bird}.")
print(f"Probability it's a bird: {probs[0]:.4f}")

In [33]:
learn.export()

In [34]:
path = Path()
path.ls(file_exts='.pkl')

In [35]:
learn_inf = load_learner(path/'export.pkl')

In [41]:
learn_inf.predict('./bird.jpg')

In [42]:
learn_inf.dls.vocab

In [45]:
btn_upload = widgets.FileUpload()
btn_upload

In [46]:
img = PILImage.create(btn_upload.data[-1])

In [48]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128)) 
out_pl

In [49]:
pred,pred_idx,probs = learn_inf.predict(img)

In [51]:
lbl_pred = widgets.Label()


In [53]:
lbl_pred.value = f'Prediction: {pred}; Probability:{probs[pred_idx]:.04f}'
lbl_pred


In [54]:
btn_run = widgets.Button(description='Classify')
btn_run

In [57]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability:{probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)

In [62]:
 VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])

In [2]:
!pip install voila

 

In [4]:
!jupyter serverextension enable voila

In [5]:
import os
import math
working_directory = os.getcwd()
print(working_directory)